In [1]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import math
import numpy as np

In [2]:
iris=datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [3]:
def cal_boundary(x,y,f):
  
    x_new=x[x[:,f].argsort()]
    values_of_classes=y[x[:,f].argsort()]
    #print("val of class ",values_of_classes)
    count_dict_all_classes=np.array([0,0,0]);
    for i in values_of_classes:
        if i ==0:
            count_dict_all_classes[0]+=1;
        elif i==1:
            count_dict_all_classes[1]+=1;
        else:
            count_dict_all_classes[2]+=1;
    print("count_dict_all_classes ",count_dict_all_classes)
    gini_node=1-(((count_dict_all_classes[0])/np.sum(count_dict_all_classes))**2)-(((count_dict_all_classes[1])/np.sum(count_dict_all_classes))**2)-(((count_dict_all_classes[2])/np.sum(count_dict_all_classes))**2)
    improvement_in_gini=0;
    boundary=0;
    for i in range(((x.shape[0]))-1):
        mid=(x_new[i,f]+x_new[i+1,f])/2;
        class0_vals=len(y[x_new[:,f]<=mid]==0);
        class1_vals=len(y[x_new[:,f]<=mid]==1);
        class2_vals=len(y[x_new[:,f]<=mid]==2);
        class01_vals=len(y[x_new[:,f]>mid]==0);
        class11_vals=len(y[x_new[:,f]>mid]==1);
        class21_vals=len(y[x_new[:,f]>mid]==2);
        D=class0_vals+class1_vals+class2_vals+class01_vals+class11_vals+class21_vals;
        D1=class0_vals+class1_vals+class2_vals;
        D2=class01_vals+class11_vals+class21_vals;
        if D1!=0:
            p0=class0_vals/D1;
            p1=class1_vals/D1;
            p2=class2_vals/D1;
        else:
            p0=0; p1=0;p2=0;
        if D2!=0:
            p00=class01_vals/D2;
            p01=class11_vals/D2;
            p02=class21_vals/D2;
        else:
            p00=0;p01=0;p02=0;
        gini_after_split=1-((p0)**2)-((p1)**2)-((p2)**2)-((p00)**2)-((p01)**2)-((p02)**2);
        if((gini_node-gini_after_split)>improvement_in_gini):
            boundary=mid;
            improvement_in_gini=gini_node-gini_after_split;
    return boundary;

In [4]:
def entropy(x,y,f):
    c1=np.count_nonzero(y.flatten()==1);
    c2=np.count_nonzero(y.flatten()==2);
    c0=y.shape[0]-c1-c2;
    if(c0+c1+c2==0):
        return 0;
    p0=c0/(c0+c1+c2);
    p1=c1/(c0+c1+c2);
    p2=c2/(c0+c1+c2);
    entropy=-((p1)*(math.log10(p1)))-(p2*(math.log10(p2)))-(p0*(math.log10(p0)));
    return entropy;

In [5]:
def gain(x,y,f):
    boundary=cal_boundary(x,y,f);
    GI_before_split=gini_index(x,y,f);
    D=x.shape[0];
    if D==0:
        return 0,boundary;
    D1=len(x[x[:,f]<=boundary]);
    D2=len(x[x[:,f]>boundary]);
    GI_after_split=(D1/D)*(gini_index(x[x[:,f]<=boundary],y[x[:,f]<=boundary],f))+(gini_index(x[x[:,f]>boundary],y[x[:,f]>boundary],f))*(D2/D);
    #entropy=entropy(no_of_c1_vals,no_of_c2_vals);
    return (GI_before_split-GI_after_split),boundary;

In [6]:
def gini_index(x,y,sf):
    
    class1_val=np.count_nonzero(y.flatten()==1);
    class2_val=np.count_nonzero(y.flatten()==2);
    class0_val=y.shape[0]-class1_val-class2_val;# is len(y[x[:,sf]]==0) correct?
    if(class0_val+class1_val+class2_val==0):
        return 0;
    p0=class0_val/(class0_val+class1_val+class2_val);
    p1=class1_val/(class0_val+class1_val+class2_val);
    p2=class2_val/(class0_val+class1_val+class2_val);
    #print(1-((p0)**2)-((p1)**2)-(p2**2))
    return (1-((p0)**2)-((p1)**2)-(p2**2));

In [7]:
def decisionTreeClassifier_Helper(x,y,feature_indices,level):
    
    if(len(set(y.flatten()))==1):
        
        count1=np.count_nonzero(y.flatten()==1);
        count2=np.count_nonzero(y.flatten()==2);
        count0=y.shape[0]-count1-count2;
        print("Level ",level)
        if(count0!=0):
            print("Count of 0 = ",count0)
        if(count1!=0):
            print("Count of 1 = ",count1)
        if(count2!=0):
            print("Count of 2 = ",count2)
        print("Current Entropy is = 0")
        print("Reached leaf node")
        
        return 0;
    elif(len(feature_indices)==0):
        return 0;
    max_gain=0;
    final_selected_feature=0;
    fs_boundary=0
    i=0
    cnt=0
    for f in feature_indices:
        curr_gain,boundary=gain(x,y,f);
        if(max_gain<curr_gain):
            max_gain=curr_gain;
            final_selected_feature=f;
            fs_boundary=boundary;
            cnt=i
        i=i+1
    #printing the count values for the splits
    feature_indices=np.delete(feature_indices,cnt)
    print("Feature indices are ",feature_indices)
    GI_node=gini_index(x,y,final_selected_feature);
    count1=np.count_nonzero(y.flatten()==1);
    count2=np.count_nonzero(y.flatten()==2);
    count0=y.shape[0]-count1-count2;
    print("Level ",level)
    print("Count of 0 = ",count0)
    print("Count of 1 = ",count1)
    print("Count of 2 = ",count2)
    print("Current Entropy is = ",entropy(x,y,final_selected_feature))
    print("Splitting on feature X",final_selected_feature," with gini-index ")
    print("Selected boundary is ",fs_boundary)
    print(GI_node)
    
    decisionTreeClassifier_Helper(x[x[:,final_selected_feature]<=fs_boundary],y[x[:,final_selected_feature]<=fs_boundary],feature_indices,level+1);
    decisionTreeClassifier_Helper(x[x[:,final_selected_feature]>fs_boundary],y[x[:,final_selected_feature]>fs_boundary],feature_indices,level+1);
    if (cnt>=len(feature_indices)):
        feature_indices=np.append(feature_indices,final_selected_feature)
    else:
        feature_indices=np.insert(feature_indices,final_selected_feature,cnt)
    
    return 0;

In [8]:
# starting to construct decision tree
def decisionTreeClassifier(x,y,feature_indices):
    #checking base cases
    print("hi")
    feature_indices_arr=np.array([i for i in range(feature_indices)])
    print(feature_indices_arr)
    decisionTreeClassifier_Helper(x,y,feature_indices_arr,0)

In [9]:
def main():
    x=iris.data;
    y=iris.target;
    decisionTreeClassifier(x,y,x.shape[1])

In [10]:
main()

hi
[0 1 2 3]
count_dict_all_classes  [50 50 50]
count_dict_all_classes  [50 50 50]
count_dict_all_classes  [50 50 50]
count_dict_all_classes  [50 50 50]
Feature indices are  [0 1 2]
Level  0
Count of 0 =  50
Count of 1 =  50
Count of 2 =  50
Current Entropy is =  0.4771212547196624
Splitting on feature X 3  with gini-index 
Selected boundary is  0.1
0.6666666666666665
Level  1
Count of 0 =  6
Current Entropy is = 0
Reached leaf node
count_dict_all_classes  [44 50 50]
count_dict_all_classes  [44 50 50]
count_dict_all_classes  [44 50 50]
Feature indices are  [1 2]
Level  1
Count of 0 =  44
Count of 1 =  50
Count of 2 =  50
Current Entropy is =  0.476356115713346
Splitting on feature X 0  with gini-index 
Selected boundary is  4.4
0.6655092592592592
Level  2
Count of 0 =  3
Current Entropy is = 0
Reached leaf node
count_dict_all_classes  [41 50 50]
count_dict_all_classes  [41 50 50]
Feature indices are  [1]
Level  2
Count of 0 =  41
Count of 1 =  50
Count of 2 =  50
Current Entropy is =  